# Deciscion Tree & Hyperopt Example

Example showing how to use the Hyperopt library (http://hyperopt.github.io) for Bayesian hyperparameter optimization (via tree of parzen estimator)

In [1]:
%load_ext watermark
%watermark -p scikit-learn,hyperopt

scikit-learn: 1.0
hyperopt    : 0.2.5



## Dataset

In [2]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn import datasets


data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

X_train_sub, X_valid, y_train_sub, y_valid = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

print('Train/Valid/Test sizes:', y_train.shape[0], y_valid.shape[0], y_test.shape[0])

Train/Valid/Test sizes: 398 80 171


## Hyperopt

In [3]:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
import hyperopt.pyll.stochastic

Some random sampling examples:

In [4]:
hyperopt.pyll.stochastic.sample(hp.loguniform('test', 1e-5, 1)) # range e^{low} to e^{high}

1.8925662130833578

In [5]:
hyperopt.pyll.stochastic.sample(hp.qloguniform('test', 1e-5, 1, 0.1)) # rounded to 0.1

1.1

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np



params =  {
    'min_samples_split': hp.choice('min_samples_split', np.arange(2, 10)),
    'min_impurity_decrease': hp.quniform('min_impurity_decrease', 0.0, 0.5, 0.05),
    'max_depth': hp.choice('max_depth', [6, 16, None])
}



def optimization_objective(params):


    tree = DecisionTreeClassifier(random_state=123, **params)
    tree.fit(X_train, y_train)
    
    accuracies = cross_val_score(
        estimator=tree, X=X_train, y=y_train, cv=10, n_jobs=-1)

    score = accuracies.mean()

    return {'loss':1-score, 'status': STATUS_OK}

In [7]:
trials = Trials()
best = fmin(fn=optimization_objective,
            space=params,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

100%|████████| 50/50 [00:01<00:00, 32.09trial/s, best loss: 0.06756410256410261]


In [8]:
best

{'max_depth': 2, 'min_impurity_decrease': 0.0, 'min_samples_split': 5}

- Attention, `fmin` returns results from `hp.choice` as an index!

In [9]:
from hyperopt import space_eval

best_params = space_eval(params, best)
print(best_params)

{'max_depth': None, 'min_impurity_decrease': 0.0, 'min_samples_split': 7}


In [10]:
tree = DecisionTreeClassifier(random_state=123, **best_params)
tree.fit(X_train, y_train)

DecisionTreeClassifier(min_samples_split=7, random_state=123)

In [11]:
print(f"Training Accuracy: {tree.score(X_train, y_train):0.2f}")
print(f"Test Accuracy: {tree.score(X_test, y_test):0.2f}")

Training Accuracy: 0.99
Test Accuracy: 0.94
